In [1]:
%tensorflow_version 1.x   #Using tensorflow 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':      #use GPU for fast process
  raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x   #Using tensorflow 1.x`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [2]:
print(tf.__version__)

1.15.2


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import requests  
file_url = "http://nlp.stanford.edu/data/glove.6B.zip"          #Download glove here is the link
    
r = requests.get(file_url, stream = True)  
  
with open("glove.6B.zip", "wb") as file:  
    for block in r.iter_content(chunk_size = 1024): 
         if block:  
             file.write(block) 

In [5]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.


In [6]:
cd '/content/go'      #downloaded in server but not in my drive it is fast to access

[Errno 2] No such file or directory: '/content/go #downloaded in server but not in my drive it is fast to access'
/content


In [7]:
!unzip  '/content/glove.6B.zip' -d go

Archive:  /content/glove.6B.zip
  inflating: go/glove.6B.50d.txt     
  inflating: go/glove.6B.100d.txt    
  inflating: go/glove.6B.200d.txt    
  inflating: go/glove.6B.300d.txt    


In [8]:
'''import numpy as np
np.random.seed(42)
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import keras
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D,GlobalAveragePooling1D
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
import warnings
warnings.filterwarnings('ignore')'''
import numpy as np
import pandas as pd
import re
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [162]:
EMBEDDING_FILE = '/content/go/glove.6B.300d.txt'      
train = pd.read_csv('/content/drive/My Drive/Hate Speech/english_dataset.tsv', sep = '\t', encoding="latin-1")
test = pd.read_csv('/content/drive/My Drive/Hate Speech/hasoc2019_en_test-2919.tsv', sep = '\t', encoding="latin-1")

In [163]:
train=train[train.task_2!="NONE"]       #drop none
test=test[test.task_2!="NONE"]
train=train.reset_index(drop=True)    #reset index
test=test.reset_index(drop=True)
X_train = train["text"]
y_train = train["task_2"]       #For task 2
X_test = test["text"]
y_test = test["task_2"]

In [164]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [165]:
#Remove stopwords from the text
stop_words = set(stopwords.words('english')) 
ix=0
for i in X_train:
  word_tokens = (i[0].split())  
  for w in word_tokens: 
    if w not in stop_words: 
        X_train[ix]=X_train[ix]+(w) 
  ix=ix+1
ix=0
for i in X_test:
  word_tokens = (i[0].split())  
  for w in word_tokens: 
    if w not in stop_words: 
        X_test[ix]= X_test[ix]+(w) 
  ix=ix+1

In [166]:
#remove punctuations
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, '')
    return x

In [167]:
ix=0
for i in X_train:
  X_train[ix]=clean_text(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=clean_text(i)
  ix=ix+1

In [168]:
#clean the numbers
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [169]:
ix=0
for i in X_train:
  X_train[ix]=clean_numbers(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=clean_numbers(i)
  ix=ix+1

In [170]:
#remove contractions
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)


In [171]:
ix=0
for i in X_train:
  X_train[ix]=replace_contractions(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=replace_contractions(i)
  ix=ix+1

In [172]:
#Stemming
'''Stemming is the process of converting words to their base forms using crude Heuristic rules.
   For example, one rule could be to remove ’s’ from the end of any word, so that ‘cats’ becomes ‘cat’. 
   or another rule could be to replace ‘ies’ with ‘i’ so that ‘ponies becomes ‘poni’.'''
from nltk.stem import  SnowballStemmer
from nltk.tokenize.toktok import ToktokTokenizer
def stem_text(text):
    tokenizer = ToktokTokenizer()
    stemmer = SnowballStemmer('english')
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

In [173]:
ix=0
for i in X_train:
  X_train[ix]=stem_text(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=stem_text(i)
  ix=ix+1

In [174]:
type(X_train)

pandas.core.series.Series

In [175]:
#After removing or cleaning the text
max_features = 11051   #No. of different words in total text 5852, 2261
maxlen = 70           #Padding length
embed_size = 300      #No. of dimensions of glove

threshold = 0.35

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [176]:
y_train = y_train.map({'HATE':0, 'OFFN': 1,'PRFN':2})    #Map accordingly 
y_test = y_test.map({'HATE':0, 'OFFN': 1,'PRFN':2})

In [177]:
y_train=keras.utils.to_categorical(y_train, num_classes=3)  #one hot encoding
y_test=keras.utils.to_categorical(y_test, num_classes=3)


In [178]:
y=[]
for i in y_train:
 y.append(np.argmax(i))

In [179]:
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [180]:
num_filters = 42
filter_sizes = [1,2,3,5]  #ngrams

In [181]:
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D,LSTM,SimpleRNN,GRU
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers


from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *

def model_train_cv(x_train,y_train,nfold,model_obj):
    splits = list(StratifiedKFold(n_splits=nfold, shuffle=True, random_state=SEED).split(x_train, y_train))
    x_train = x_train
    y_train = np.array(y_train)
    # matrix for the out-of-fold predictions
    train_oof_preds = np.zeros((x_train.shape[0]))
    for i, (train_idx, valid_idx) in enumerate(splits):
        print(f'Fold {i + 1}')
        x_train_fold = x_train[train_idx.astype(int)]
        y_train_fold = y_train[train_idx.astype(int)]
        x_val_fold = x_train[valid_idx.astype(int)]
        y_val_fold = y_train[valid_idx.astype(int)]
        # Changed it here a little bit since the custom attention layer is not getting deepcopy
        clf = model_obj
        clf.load_weights('model.h5')
        clf.fit(x_train_fold, y_train_fold, batch_size=512, epochs=5, validation_data=(x_val_fold, y_val_fold))
        
        valid_preds_fold = clf.predict(x_val_fold)[:,0]

        # storing OOF predictions
        train_oof_preds[valid_idx] = valid_preds_fold
    return train_oof_preds

In [182]:
#model of bi-lstm with attention
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


def model_lstm_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(3, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [183]:
#model of lstm with attention
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


def model_slstm_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(3, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [184]:
#model of gru with attention
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


def model_gru_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    x = Bidirectional(GRU(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(3, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [185]:
#model of Simple RNN with attention
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


def model_rnn_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(SimpleRNN(128, return_sequences=True))(x)
    x = Bidirectional(SimpleRNN(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(3, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [186]:
print(tf.__version__)

1.15.2


In [187]:
ls_model = model_lstm_atten(embedding_matrix) #model of bi-lstm with attention

In [188]:
sls_model = model_slstm_atten(embedding_matrix) #model of lstm with attention

In [189]:
gru_model = model_gru_atten(embedding_matrix) #model of gru with attention

In [190]:
rnn_model = model_rnn_atten(embedding_matrix) #model of rnn with attention

In [191]:
batch_size = 256
epochs = 10

X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95,
                                              random_state=42) #split this into train and validation

In [192]:
ls_hist = ls_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)      #Train bi-lstm with attention

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 3s - loss: 1.0411 - accuracy: 0.4714 - val_loss: 0.9898 - val_accuracy: 0.5439
Epoch 2/10
 - 1s - loss: 0.9789 - accuracy: 0.5347 - val_loss: 1.0198 - val_accuracy: 0.5789
Epoch 3/10
 - 1s - loss: 0.9184 - accuracy: 0.5706 - val_loss: 0.9656 - val_accuracy: 0.5614
Epoch 4/10
 - 1s - loss: 0.8417 - accuracy: 0.6064 - val_loss: 0.9906 - val_accuracy: 0.5526
Epoch 5/10
 - 1s - loss: 0.8165 - accuracy: 0.6032 - val_loss: 0.9702 - val_accuracy: 0.5614
Epoch 6/10
 - 1s - loss: 0.7721 - accuracy: 0.6516 - val_loss: 0.9719 - val_accuracy: 0.5526
Epoch 7/10
 - 1s - loss: 0.7192 - accuracy: 0.6670 - val_loss: 1.0235 - val_accuracy: 0.4912
Epoch 8/10
 - 1s - loss: 0.6755 - accuracy: 0.6959 - val_loss: 1.0840 - val_accuracy: 0.5088
Epoch 9/10
 - 1s - loss: 0.6557 - accuracy: 0.7117 - val_loss: 1.0983 - val_accuracy: 0.5000
Epoch 10/10
 - 1s - loss: 0.6315 - accuracy: 0.7094 - val_loss: 1.1282 - val_accuracy: 0.4912


In [193]:
sls_hist = sls_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)    #Train lstm with attention

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 5s - loss: 1.0432 - accuracy: 0.4751 - val_loss: 0.9924 - val_accuracy: 0.5263
Epoch 2/10
 - 3s - loss: 0.9937 - accuracy: 0.5105 - val_loss: 0.9718 - val_accuracy: 0.5263
Epoch 3/10
 - 3s - loss: 0.9327 - accuracy: 0.5599 - val_loss: 0.9871 - val_accuracy: 0.5351
Epoch 4/10
 - 3s - loss: 0.8683 - accuracy: 0.5855 - val_loss: 1.0868 - val_accuracy: 0.4737
Epoch 5/10
 - 3s - loss: 0.8471 - accuracy: 0.6092 - val_loss: 0.9684 - val_accuracy: 0.5614
Epoch 6/10
 - 3s - loss: 0.7791 - accuracy: 0.6465 - val_loss: 0.9715 - val_accuracy: 0.5702
Epoch 7/10
 - 3s - loss: 0.7634 - accuracy: 0.6558 - val_loss: 0.9727 - val_accuracy: 0.5965
Epoch 8/10
 - 3s - loss: 0.7166 - accuracy: 0.6800 - val_loss: 1.0157 - val_accuracy: 0.5702
Epoch 9/10
 - 3s - loss: 0.6720 - accuracy: 0.6986 - val_loss: 1.0544 - val_accuracy: 0.5351
Epoch 10/10
 - 3s - loss: 0.6207 - accuracy: 0.7308 - val_loss: 1.1176 - val_accuracy: 0.5351


In [194]:
gru_hist = gru_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)      #Train GRU with attention

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 6s - loss: 1.0518 - accuracy: 0.4686 - val_loss: 0.9878 - val_accuracy: 0.5351
Epoch 2/10
 - 4s - loss: 0.9418 - accuracy: 0.5701 - val_loss: 0.9653 - val_accuracy: 0.5789
Epoch 3/10
 - 3s - loss: 0.8746 - accuracy: 0.5878 - val_loss: 0.9614 - val_accuracy: 0.5351
Epoch 4/10
 - 4s - loss: 0.8284 - accuracy: 0.6055 - val_loss: 0.9450 - val_accuracy: 0.5439
Epoch 5/10
 - 3s - loss: 0.7958 - accuracy: 0.6274 - val_loss: 0.9454 - val_accuracy: 0.4912
Epoch 6/10
 - 3s - loss: 0.7710 - accuracy: 0.6414 - val_loss: 0.9658 - val_accuracy: 0.5789
Epoch 7/10
 - 3s - loss: 0.7241 - accuracy: 0.6679 - val_loss: 1.0017 - val_accuracy: 0.5702
Epoch 8/10
 - 4s - loss: 0.6820 - accuracy: 0.6833 - val_loss: 1.0097 - val_accuracy: 0.5088
Epoch 9/10
 - 4s - loss: 0.6238 - accuracy: 0.7233 - val_loss: 1.0869 - val_accuracy: 0.5526
Epoch 10/10
 - 3s - loss: 0.5815 - accuracy: 0.7438 - val_loss: 1.1702 - val_accuracy: 0.5088


In [195]:
rnn_hist = rnn_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)      #Train RNN with attention

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 4s - loss: 1.0304 - accuracy: 0.4732 - val_loss: 0.9783 - val_accuracy: 0.5526
Epoch 2/10
 - 2s - loss: 0.9454 - accuracy: 0.5491 - val_loss: 0.9436 - val_accuracy: 0.5614
Epoch 3/10
 - 2s - loss: 0.8329 - accuracy: 0.6102 - val_loss: 0.9278 - val_accuracy: 0.5526
Epoch 4/10
 - 2s - loss: 0.7121 - accuracy: 0.6861 - val_loss: 1.0575 - val_accuracy: 0.4737
Epoch 5/10
 - 2s - loss: 0.6061 - accuracy: 0.7392 - val_loss: 1.1040 - val_accuracy: 0.4649
Epoch 6/10
 - 2s - loss: 0.4891 - accuracy: 0.8225 - val_loss: 1.2006 - val_accuracy: 0.4737
Epoch 7/10
 - 2s - loss: 0.3674 - accuracy: 0.8701 - val_loss: 1.3341 - val_accuracy: 0.5175
Epoch 8/10
 - 2s - loss: 0.2843 - accuracy: 0.9068 - val_loss: 1.4312 - val_accuracy: 0.5175
Epoch 9/10
 - 2s - loss: 0.2180 - accuracy: 0.9353 - val_loss: 1.5921 - val_accuracy: 0.4649
Epoch 10/10
 - 2s - loss: 0.1608 - accuracy: 0.9530 - val_loss: 1.6547 - val_accuracy: 0.5175


In [196]:
from sklearn.metrics import classification_report

In [197]:
y_t=[]
for i in y_test:
  y_t.append(np.argmax(i))    #Using argmax to find the index of one hot encoding

In [198]:
y_ls_pred = ls_model.predict(x_test)      #predict y_test by bi-lstm with attention
y_ls_p=[]
for i in y_ls_pred:
  y_ls_p.append(np.argmax(i))


In [199]:
print('Model = Attention Bidirectonal LSTM\n')  
print(classification_report(y_t, y_ls_p))

Model = Attention Bidirectonal LSTM

              precision    recall  f1-score   support

           0       0.67      0.60      0.63       124
           1       0.60      0.13      0.21        71
           2       0.52      0.90      0.66        93

    accuracy                           0.58       288
   macro avg       0.60      0.54      0.50       288
weighted avg       0.60      0.58      0.54       288



In [200]:
y_sls_pred = sls_model.predict(x_test)  #predict y_test by lstm with attention
y_sls_p=[]
for i in y_sls_pred:
  y_sls_p.append(np.argmax(i))

In [201]:
print('Model = Attention LSTM\n')
print(classification_report(y_t, y_sls_p))

Model = Attention LSTM

              precision    recall  f1-score   support

           0       0.62      0.81      0.70       124
           1       0.64      0.13      0.21        71
           2       0.68      0.81      0.74        93

    accuracy                           0.64       288
   macro avg       0.65      0.58      0.55       288
weighted avg       0.64      0.64      0.59       288



In [202]:
y_gru_pred = gru_model.predict(x_test)    #predict y_test by GRU with attention
y_gru_p=[]
for i in y_gru_pred:
  y_gru_p.append(np.argmax(i))

In [203]:
print('Model = Attention GRU\n')
print(classification_report(y_t, y_gru_p))

Model = Attention GRU

              precision    recall  f1-score   support

           0       0.61      0.84      0.71       124
           1       0.58      0.10      0.17        71
           2       0.71      0.81      0.76        93

    accuracy                           0.65       288
   macro avg       0.64      0.58      0.54       288
weighted avg       0.64      0.65      0.59       288



In [204]:
y_rnn_pred = rnn_model.predict(x_test)    #predict y_test by RNN with attention
y_rnn_p=[]
for i in y_rnn_pred:
  y_rnn_p.append(np.argmax(i))

In [205]:
print('Model = Attention SimpleRNN\n')
print(classification_report(y_t, y_rnn_p))

Model = Attention SimpleRNN

              precision    recall  f1-score   support

           0       0.54      0.63      0.58       124
           1       0.38      0.21      0.27        71
           2       0.63      0.71      0.67        93

    accuracy                           0.55       288
   macro avg       0.52      0.52      0.51       288
weighted avg       0.53      0.55      0.53       288

